##1. Instalando bibliotecas


In [1]:
 !pip install 'fsspec>=0.3.3'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 4.3 MB/s 


##2. Importando bibliotecas e arquivos do drive


In [2]:
import dask.dataframe as dd
import plotly.express as px
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/olist_geoleocated_encoder.csv')

In [5]:
df = df.compute()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112660 entries, 0 to 112659
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       112660 non-null  int64  
 1   order_item_id                  112660 non-null  float64
 2   product_id                     112660 non-null  int64  
 3   seller_id                      112660 non-null  int64  
 4   shipping_limit_date            112660 non-null  int64  
 5   price                          112660 non-null  float64
 6   freight_value                  112660 non-null  float64
 7   payment_sequential             112660 non-null  float64
 8   payment_type                   112660 non-null  int64  
 9   payment_installments           112660 non-null  float64
 10  payment_value                  112660 non-null  float64
 11  review_id                      112660 non-null  int64  
 12  review_score                  

In [7]:
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,costumer_lat,costumer_lng,seller_lat,seller_lng
0,0,1.0,8216,812,22606,58.9,13.29,1.0,1,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-21.763186,-41.310265,-22.497188,-44.127324
1,7014,1.0,8216,812,13182,55.9,17.96,1.0,0,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-17.884803,-51.717167,-22.497188,-44.127324
2,30559,1.0,8216,812,69555,64.9,18.33,1.0,1,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-19.913964,-43.976106,-22.497188,-44.127324
3,40954,1.0,8216,812,17142,58.9,16.17,1.0,1,3.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-25.520334,-49.124988,-22.497188,-44.127324
4,45875,1.0,8216,812,18388,58.9,13.29,1.0,1,4.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-20.655379,-43.776331,-22.497188,-44.127324


##3. Calculando tempo de entrega


In [8]:
#transformando em datas pelo pandas
for col in df[['review_creation_date','order_approved_at','order_delivered_carrier_date','order_estimated_delivery_date','shipping_limit_date','review_answer_timestamp','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date']]:
    df[col] = pd.to_datetime(df[col])

In [9]:
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,costumer_lat,costumer_lng,seller_lat,seller_lng
0,0,1.0,8216,812,1970-01-01 00:00:00.000022606,58.9,13.29,1.0,1,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-21.763186,-41.310265,-22.497188,-44.127324
1,7014,1.0,8216,812,1970-01-01 00:00:00.000013182,55.9,17.96,1.0,0,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-17.884803,-51.717167,-22.497188,-44.127324
2,30559,1.0,8216,812,1970-01-01 00:00:00.000069555,64.9,18.33,1.0,1,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-19.913964,-43.976106,-22.497188,-44.127324
3,40954,1.0,8216,812,1970-01-01 00:00:00.000017142,58.9,16.17,1.0,1,3.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-25.520334,-49.124988,-22.497188,-44.127324
4,45875,1.0,8216,812,1970-01-01 00:00:00.000018388,58.9,13.29,1.0,1,4.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-20.655379,-43.776331,-22.497188,-44.127324



*   Calculando o tempo, em dias, que o produto levou para ser entregue

*   Para realizar o calculo será feito a subtração do dia em que o pedido foi entregue menos o dia em que o pedido foi confirmado (order_delivered_carrier_date - order_approved_at) 



In [10]:
#funcao que calcula o tempo da entrega
def days_to_delivery(data):
    # verifica se a entrega foi no mesmo mes 
    if data['order_delivered_carrier_date'].month == data['order_approved_at'].month:
        if data['order_delivered_carrier_date'].day < data['order_approved_at'].day:
            return np.nan #se o dia da entrega é anterior ao dia do pedido -> ERRO
        else: 
            return data['order_delivered_carrier_date'].day - data['order_approved_at'].day
    
    # se ocorreu em meses diferentes, calcula-se os dias para cada mes e depois soma
    elif data['order_delivered_carrier_date'].month != data['order_approved_at'].month:
        if data['order_delivered_carrier_date'].month < data['order_approved_at'].month:
            return np.nan
        else:
            return (pd.Period(data['order_approved_at'], freq='D').days_in_month - data['order_approved_at'].day) + data['order_delivered_carrier_date'].day

    
df['days_to_delivery'] = df.apply(days_to_delivery, axis=1)

In [11]:
df.head(10)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,costumer_lat,costumer_lng,seller_lat,seller_lng,days_to_delivery
0,0,1.0,8216,812,1970-01-01 00:00:00.000022606,58.9,13.29,1.0,1,2.0,...,4.0,650.0,28.0,9.0,14.0,-21.763186,-41.310265,-22.497188,-44.127324,0
1,7014,1.0,8216,812,1970-01-01 00:00:00.000013182,55.9,17.96,1.0,0,1.0,...,4.0,650.0,28.0,9.0,14.0,-17.884803,-51.717167,-22.497188,-44.127324,0
2,30559,1.0,8216,812,1970-01-01 00:00:00.000069555,64.9,18.33,1.0,1,2.0,...,4.0,650.0,28.0,9.0,14.0,-19.913964,-43.976106,-22.497188,-44.127324,0
3,40954,1.0,8216,812,1970-01-01 00:00:00.000017142,58.9,16.17,1.0,1,3.0,...,4.0,650.0,28.0,9.0,14.0,-25.520334,-49.124988,-22.497188,-44.127324,0
4,45875,1.0,8216,812,1970-01-01 00:00:00.000018388,58.9,13.29,1.0,1,4.0,...,4.0,650.0,28.0,9.0,14.0,-20.655379,-43.776331,-22.497188,-44.127324,0
5,58794,1.0,8216,812,1970-01-01 00:00:00.000016757,55.9,26.93,1.0,1,1.0,...,4.0,650.0,28.0,9.0,14.0,-12.906138,-39.192076,-22.497188,-44.127324,0
6,69027,1.0,8216,812,1970-01-01 00:00:00.000056807,64.9,38.50,1.0,0,1.0,...,4.0,650.0,28.0,9.0,14.0,-15.036469,-41.260324,-22.497188,-44.127324,0
7,74190,2.0,8216,812,1970-01-01 00:00:00.000017990,58.9,18.12,1.0,0,1.0,...,4.0,650.0,28.0,9.0,14.0,-17.782743,-50.917956,-22.497188,-44.127324,0
8,74190,1.0,8359,812,1970-01-01 00:00:00.000017990,58.9,17.83,1.0,0,1.0,...,6.0,530.0,30.0,9.0,14.0,-17.782743,-50.917956,-22.497188,-44.127324,0
9,82148,1.0,8216,812,1970-01-01 00:00:00.000010776,55.9,35.71,2.0,3,1.0,...,4.0,650.0,28.0,9.0,14.0,-7.957598,-34.874754,-22.497188,-44.127324,0


In [12]:
#baixando csv com a coluna a ser prevista
from google.colab import files
df.to_csv('olist_geoleocated_time_prediction.csv',index=False) 
files.download('olist_geoleocated_time_prediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>